In [37]:
!pip install datasets

In [2]:
#importing libraries
import pandas as pd
import numpy as np
import os
# from google.colab import drive
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from transformers import EvalPrediction

In [5]:
!unzip merged.zip

Archive:  merged.zip
   creating: Merged/
   creating: Merged/subtask-2-documents/
  inflating: Merged/subtask-2-documents/EN_UA_300127.txt  
  inflating: Merged/subtask-2-documents/EN_CC_200145.txt  
  inflating: Merged/subtask-2-documents/EN_UA_103372.txt  
  inflating: Merged/subtask-2-documents/EN_CC_200150.txt  
  inflating: Merged/subtask-2-documents/EN_CC_100232.txt  
  inflating: Merged/subtask-2-documents/EN_CC_200013.txt  
  inflating: Merged/subtask-2-documents/EN_UA_026740.txt  
  inflating: Merged/subtask-2-documents/EN_UA_010735.txt  
  inflating: Merged/subtask-2-documents/EN_UA_300154.txt  
  inflating: Merged/subtask-2-documents/EN_UA_004343.txt  
  inflating: Merged/subtask-2-documents/EN_UA_010909.txt  
  inflating: Merged/subtask-2-documents/EN_UA_022339.txt  
  inflating: Merged/subtask-2-documents/EN_UA_DEV_100029.txt  
  inflating: Merged/subtask-2-documents/EN_CC_100169.txt  
  inflating: Merged/subtask-2-documents/EN_UA_300134.txt  
  inflating: Merged/subtask-

In [8]:
file_path = "Merged/concatenated_data.txt"
df = pd.read_csv(file_path, sep="\t", header=None, names=["Document_ID", "High_Level_Narratives", "Sub_Narratives"])

In [9]:
df

,Document_ID,High_Level_Narratives,Sub_Narratives
0,EN_CC_100013.txt,CC: Criticism of climate movement,CC: Criticism of climate movement: Ad hominem ...
1,EN_UA_300009.txt,Other,Other
2,EN_UA_300017.txt,Other,Other
3,EN_CC_100021.txt,Other,Other
4,EN_UA_300041.txt,Other,Other
...,...,...,...
435,EN_CC_200069.txt,CC: Criticism of climate policies;CC: Criticis...,CC: Criticism of climate policies: Other;CC: C...
436,EN_UA_DEV_100019.txt,Other,Other
437,EN_CC_200078.txt,Other,Other
438,EN_CC_200065.txt,CC: Criticism of climate movement;CC: Question...,CC: Criticism of climate movement: Climate mov...


In [10]:
# Group by Document_ID and create lists of narratives
df = df.groupby("Document_ID").agg({
    "High_Level_Narratives": lambda x: list(set(";".join(x).split(";"))),
    "Sub_Narratives": lambda x: list(set(";".join(x).split(";")))
}).reset_index()

df.rename(columns={
    "High_Level_Narratives": "High_Level_Narratives_List",
    "Sub_Narratives": "Sub_Narratives_List"
}, inplace=True)


In [11]:
documents_folder = 'Merged/subtask-2-documents'


def load_documents(folder_path):
    documents = {}
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):  # Ensure only text files are read
            doc_id = filename  # Extract Document_ID from filename
            with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as file:
                documents[doc_id] = file.read().strip()
    return documents

# Load document texts into a dictionary
document_texts = load_documents(documents_folder)

# Map document texts to the DataFrame
df['Text'] = df['Document_ID'].map(document_texts)

# Check if all documents are successfully mapped
print(f"Number of missing documents: {df['Text'].isnull().sum()}")
df


Number of missing documents: 0


,Document_ID,High_Level_Narratives_List,Sub_Narratives_List,Text
0,EN_CC_100000.txt,"[CC: Controversy about green technologies, CC:...",[CC: Hidden plots by secret schemes of powerfu...,Pentagon plans to serve LAB-GROWN MEAT to troo...
1,EN_CC_100002.txt,[CC: Hidden plots by secret schemes of powerfu...,[CC: Criticism of institutions and authorities...,Oxford Residents Mount Resistance Against the ...
2,EN_CC_100003.txt,[CC: Criticism of institutions and authorities...,[CC: Criticism of climate movement: Ad hominem...,"Fonda Heads To Canada For Oil Sands Protest, M..."
3,EN_CC_100004.txt,"[CC: Controversy about green technologies, CC:...",[CC: Criticism of climate movement: Ad hominem...,A Tesla Owner Just Exposed A Sick Secret About...
4,EN_CC_100005.txt,[CC: Criticism of climate movement],[CC: Criticism of climate movement: Climate mo...,Climate Crazies Fail in Attempt to Vandalize A...
...,...,...,...,...
435,EN_UA_DEV_216.txt,"[URW: Discrediting the West, Diplomacy, URW: N...",[URW: Negative Consequences for the West: Sanc...,"EU 'biggest loser' in Ukraine conflicts, Hunga..."
436,EN_UA_DEV_22.txt,"[URW: Discrediting the West, Diplomacy, URW: B...","[URW: Discrediting the West, Diplomacy: The EU...",European Parliament members clash over support...
437,EN_UA_DEV_23.txt,"[URW: Amplifying war-related fears, URW: Distr...",[URW: Distrust towards Media: Western media is...,What is the current trajectory of the evil emp...
438,EN_UA_DEV_24.txt,"[URW: Discrediting the West, Diplomacy, URW: N...",[URW: Negative Consequences for the West: Sanc...,Europe ‘Shot Itself in the Lungs’ With Sanctio...


In [12]:
cc_narratives = sorted(
    set(
        narrative for narratives in df['High_Level_Narratives_List']
        for narrative in narratives if narrative.startswith("CC:")
    )
)

print(len(cc_narratives))

10


In [13]:
df

,Document_ID,High_Level_Narratives_List,Sub_Narratives_List,Text
0,EN_CC_100000.txt,"[CC: Controversy about green technologies, CC:...",[CC: Hidden plots by secret schemes of powerfu...,Pentagon plans to serve LAB-GROWN MEAT to troo...
1,EN_CC_100002.txt,[CC: Hidden plots by secret schemes of powerfu...,[CC: Criticism of institutions and authorities...,Oxford Residents Mount Resistance Against the ...
2,EN_CC_100003.txt,[CC: Criticism of institutions and authorities...,[CC: Criticism of climate movement: Ad hominem...,"Fonda Heads To Canada For Oil Sands Protest, M..."
3,EN_CC_100004.txt,"[CC: Controversy about green technologies, CC:...",[CC: Criticism of climate movement: Ad hominem...,A Tesla Owner Just Exposed A Sick Secret About...
4,EN_CC_100005.txt,[CC: Criticism of climate movement],[CC: Criticism of climate movement: Climate mo...,Climate Crazies Fail in Attempt to Vandalize A...
...,...,...,...,...
435,EN_UA_DEV_216.txt,"[URW: Discrediting the West, Diplomacy, URW: N...",[URW: Negative Consequences for the West: Sanc...,"EU 'biggest loser' in Ukraine conflicts, Hunga..."
436,EN_UA_DEV_22.txt,"[URW: Discrediting the West, Diplomacy, URW: B...","[URW: Discrediting the West, Diplomacy: The EU...",European Parliament members clash over support...
437,EN_UA_DEV_23.txt,"[URW: Amplifying war-related fears, URW: Distr...",[URW: Distrust towards Media: Western media is...,What is the current trajectory of the evil emp...
438,EN_UA_DEV_24.txt,"[URW: Discrediting the West, Diplomacy, URW: N...",[URW: Negative Consequences for the West: Sanc...,Europe ‘Shot Itself in the Lungs’ With Sanctio...


In [14]:
df = df[df["High_Level_Narratives_List"].apply(lambda narratives: any("CC:" in narrative for narrative in narratives))]


In [15]:
def preprocess_multi_label(data, narratives):
    label_vectors = []
    for narratives_list in data['High_Level_Narratives_List']:
        vector = [1 if narrative in narratives_list else 0 for narrative in narratives]
        label_vectors.append(vector)
    data['Labels'] = label_vectors
    return data

In [16]:
df = preprocess_multi_label(df, cc_narratives)

<ipython-input-15-d47a499dcf72>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Labels'] = label_vectors


In [17]:
df.head()

,Document_ID,High_Level_Narratives_List,Sub_Narratives_List,Text,Labels
0,EN_CC_100000.txt,"[CC: Controversy about green technologies, CC:...",[CC: Hidden plots by secret schemes of powerfu...,Pentagon plans to serve LAB-GROWN MEAT to troo...,"[0, 0, 1, 0, 0, 1, 0, 0, 1, 0]"
1,EN_CC_100002.txt,[CC: Hidden plots by secret schemes of powerfu...,[CC: Criticism of institutions and authorities...,Oxford Residents Mount Resistance Against the ...,"[0, 0, 0, 0, 0, 1, 0, 0, 1, 0]"
2,EN_CC_100003.txt,[CC: Criticism of institutions and authorities...,[CC: Criticism of climate movement: Ad hominem...,"Fonda Heads To Canada For Oil Sands Protest, M...","[0, 0, 0, 1, 0, 1, 0, 0, 0, 0]"
3,EN_CC_100004.txt,"[CC: Controversy about green technologies, CC:...",[CC: Criticism of climate movement: Ad hominem...,A Tesla Owner Just Exposed A Sick Secret About...,"[0, 0, 1, 1, 0, 1, 0, 0, 0, 0]"
4,EN_CC_100005.txt,[CC: Criticism of climate movement],[CC: Criticism of climate movement: Climate mo...,Climate Crazies Fail in Attempt to Vandalize A...,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]"


In [18]:
def calculate_class_weights(labels):
    label_sums = np.sum(labels, axis=0)
    total_samples = len(labels)
    class_weights = [total_samples / (len(label_sums) * count) for count in label_sums]
    return torch.tensor(class_weights, dtype=torch.float)

In [19]:
labels = np.array(df['Labels'].tolist(), dtype=np.float32)
class_weights = calculate_class_weights(labels)

In [20]:
class_weights

tensor([1.7143, 2.4000, 0.5217, 0.2609, 0.2927, 0.1714, 0.6000, 1.7143, 0.3077,
        0.5217])

In [23]:
def handle_rare_classes(data, labels, min_samples=15):
    label_sums = np.sum(labels, axis=0)
    rare_classes = np.where(label_sums < min_samples)[0]

    for rare_class in rare_classes:
        rare_indices = [i for i, lbl in enumerate(labels) if lbl[rare_class] == 1]
        if len(rare_indices) > 0:
            duplicate_data = data.iloc[rare_indices]
            data = pd.concat([data] + [duplicate_data] * (min_samples - len(rare_indices)), ignore_index=True)
            labels = np.vstack([labels] + [labels[rare_indices]] * (min_samples - len(rare_indices)))
    return data, labels


In [21]:
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['Text'], labels, test_size=0.2, random_state=42)


## test shavad ham be tanhayi ham combined ba class weight

In [24]:
train_texts,train_labels = handle_rare_classes(train_texts,train_labels)


In [28]:
train_dataset = Dataset.from_dict({"text": train_texts.tolist(), "labels": train_labels.tolist()})
test_dataset = Dataset.from_dict({"text": test_texts.tolist(), "labels": test_labels.tolist()})
datasets = DatasetDict({"train": train_dataset, "test": test_dataset})


In [25]:
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
model = AutoModelForSequenceClassification.from_pretrained(
    "roberta-base", num_labels=labels.shape[1], problem_type="multi_label_classification"
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

# Tokenize the dataset
tokenized_datasets = datasets.map(tokenize_function, batched=True)


Map:   0%|          | 0/222 [00:00<?, ? examples/s]

Map:   0%|          | 0/24 [00:00<?, ? examples/s]

## zamani estefade she ke darim az class weight estefade mikonim

In [30]:
# def custom_loss_function(logits, labels, class_weights):
#     """
#     Custom loss function using BCEWithLogitsLoss and class weights.
#     """
#     # Define the loss function with class weights
#     loss_fct = torch.nn.BCEWithLogitsLoss(pos_weight=class_weights)
#     loss = loss_fct(logits, labels)  # Compute the weighted loss
#     return loss

In [38]:
from sklearn.metrics import precision_score, recall_score, f1_score
from transformers import EvalPrediction

def compute_metrics(p: EvalPrediction):
    # Apply sigmoid activation to model predictions (logits)
    sigmoid_preds = torch.sigmoid(torch.tensor(p.predictions)).numpy()

    # Convert probabilities to binary predictions using a threshold of 0.5
    preds = (sigmoid_preds > 0.5).astype(int)

    # Ground-truth labels
    labels = p.label_ids

    # Calculate weighted precision, recall, and F1-score
    precision = precision_score(labels, preds, average="weighted", zero_division=0)
    recall = recall_score(labels, preds, average="weighted", zero_division=0)
    f1 = f1_score(labels, preds, average="weighted", zero_division=0)

    # Return the computed metrics
    return {"precision": precision, "recall": recall, "f1": f1}


In [39]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [40]:

training_args = TrainingArguments(
    output_dir='./results',                    # Directory to store checkpoints and final model
    num_train_epochs=50,                       # Total number of training epochs
    learning_rate=5e-5,  # Fine-tuning learning rate
    per_device_train_batch_size=4,            # Batch size per device during training
    per_device_eval_batch_size=4,             # Batch size for evaluation
    evaluation_strategy='epoch',               # Evaluate at the end of each epoch
    save_strategy='epoch',                     # Save model at the end of each epoch
    load_best_model_at_end=True,               # Load the best model at the end of training
    metric_for_best_model='f1',                # Use F1 score to evaluate the best model
    greater_is_better=True,                    # Higher F1 is better
    logging_dir='./logs',                      # Directory for storing logs
    logging_steps=100,                         # Log every 100 steps
    save_total_limit=1,                        # Limit the total amount of checkpoints
    seed=42,
    weight_decay=0.01,# Seed for reproducibility
    # fp16=True
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [34]:
# class CustomTrainer(Trainer):
#     def __init__(self, class_weights, *args, **kwargs):
#         super().__init__(*args, **kwargs)
#         self.class_weights = class_weights  # Store class weights for later use

#     def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
#         labels = inputs.pop("labels")  # Extract labels
#         outputs = model(**inputs)  # Forward pass
#         logits = outputs.logits  # Get logits from model outputs

#         # Move labels and class weights to the same device as logits
#         device = logits.device
#         labels = labels.to(device)
#         class_weights = self.class_weights.to(device)

#         # Use your custom loss function
#         loss = custom_loss_function(logits, labels, class_weights)

#         return (loss, outputs) if return_outputs else loss


In [41]:
# # Convert class weights to a PyTorch tensor
# class_weights_tensor = torch.tensor(class_weights, dtype=torch.float)

# # Initialize the custom trainer
# trainer = CustomTrainer(
#     class_weights=class_weights_tensor,  # Pass your class weights
#     model=model,  # Your model
#     args=training_args,  # Training arguments
#     train_dataset=tokenized_datasets["train"],  # Training dataset
#     eval_dataset=tokenized_datasets["test"],  # Evaluation dataset
#     tokenizer=tokenizer,  # Tokenizer
#     compute_metrics=compute_metrics,  # Metrics function
# )

<ipython-input-41-68d6bccb169c>:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  class_weights_tensor = torch.tensor(class_weights, dtype=torch.float)
<ipython-input-41-68d6bccb169c>:5: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [42]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

<ipython-input-42-e4b1a771ab8b>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [43]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,No log,0.602901,0.690052,0.524590,0.560656
2,0.022100,0.639653,0.831446,0.540984,0.583803
3,0.022100,0.601681,0.710343,0.573770,0.603661
4,0.022600,0.537435,0.682201,0.590164,0.610169
5,0.022600,0.614405,0.731413,0.557377,0.575369
6,0.018000,0.791256,0.682745,0.491803,0.506173
7,0.018000,0.606349,0.775774,0.606557,0.639959
8,0.012000,0.651645,0.689982,0.557377,0.596142
9,0.007600,0.634907,0.695380,0.540984,0.593023
10,0.007600,0.618778,0.706673,0.524590,0.596249


TrainOutput(global_step=2800, training_loss=0.004842765493584531, metrics={'train_runtime': 1794.9259, 'train_samples_per_second': 6.184, 'train_steps_per_second': 1.56, 'total_flos': 2920742492774400.0, 'train_loss': 0.004842765493584531, 'epoch': 50.0})